# torchtext 튜토리얼_한국어

## 훈련 데이터와 테스트 데이터 다운

In [3]:
# 네이버 영화리뷰 데이터 다운
import urllib.request
import pandas as pd

urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

('ratings_test.txt', <http.client.HTTPMessage at 0x7f4a12d36850>)

In [4]:
train_df = pd.read_table('ratings_train.txt')
test_df = pd.read_table('ratings_test.txt')

In [5]:
print('훈련 데이터 샘플의 개수 : {}'.format(len(train_df)))
print('테스트 데이터 샘플의 개수 : {}'.format(len(test_df)))

훈련 데이터 샘플의 개수 : 150000
테스트 데이터 샘플의 개수 : 50000


## 필드 정의(torchtext.data)

In [6]:
! pip install konlpy

     |████████████████████████████████| 19.4 MB 1.9 MB/s 
     |████████████████████████████████| 448 kB 36.0 MB/s 


In [7]:

from torchtext.legacy import data 
from konlpy.tag import Okt

In [8]:
tokenizer = Okt()

In [9]:
# 필드 정의
ID = data.Field(sequential = False,
                use_vocab = False) # 실제 사용은 하지 않을 예정

TEXT = data.Field(sequential=True,
                  use_vocab=True,
                  tokenize=tokenizer.morphs, # 토크나이저로는 Mecab 사용.
                  lower=True,
                  batch_first=True,
                  fix_length=20)

LABEL = data.Field(sequential=False,
                   use_vocab=False,
                   is_target=True)

## 데이터셋 만들기

In [10]:
from torchtext.legacy.data import TabularDataset

train_data, test_data =TabularDataset.splits(
     path='.', train='ratings_train.txt', test='ratings_test.txt', format='tsv',
        fields=[('id', ID), ('text', TEXT), ('label', LABEL)], skip_header=True)


In [11]:
print('훈련 샘플의 개수 : {}'.format(len(train_data)))
print('테스트 샘플의 개수 : {}'.format(len(test_data)))

훈련 샘플의 개수 : 150000
테스트 샘플의 개수 : 50000


## 단어 집합 만들기

In [12]:
TEXT.build_vocab(train_data, min_freq = 10, max_size = 10000)

In [13]:
print('단어 집합의 크기 : {}'.format(len(TEXT.vocab)))

단어 집합의 크기 : 10002


In [14]:
print(TEXT.vocab.stoi)

defaultdict(<bound method Vocab._default_unk_index of <torchtext.legacy.vocab.Vocab object at 0x7f48f205d710>>, {'<unk>': 0, '<pad>': 1, '.': 2, '이': 3, '영화': 4, '의': 5, '..': 6, '가': 7, '에': 8, '을': 9, '...': 10, '도': 11, '들': 12, ',': 13, '는': 14, '를': 15, '은': 16, '너무': 17, '?': 18, '한': 19, '다': 20, '정말': 21, '적': 22, '만': 23, '!': 24, '진짜': 25, '으로': 26, '점': 27, '로': 28, '에서': 29, '연기': 30, '과': 31, '평점': 32, '것': 33, '최고': 34, '내': 35, '~': 36, '나': 37, '그': 38, '잘': 39, '와': 40, '안': 41, '인': 42, '이런': 43, '스토리': 44, '생각': 45, '못': 46, '....': 47, '왜': 48, '드라마': 49, '게': 50, '이다': 51, '감동': 52, '사람': 53, '1': 54, '하는': 55, '보고': 56, '하고': 57, '말': 58, '고': 59, '아': 60, '더': 61, '때': 62, 'ㅋㅋ': 63, '배우': 64, '거': 65, '감독': 66, '그냥': 67, '요': 68, '본': 69, '재미': 70, '내용': 71, '뭐': 72, '중': 73, '까지': 74, '!!': 75, '좀': 76, '쓰레기': 77, '보다': 78, '없는': 79, '시간': 80, '수': 81, '지': 82, '네': 83, '봤는데': 84, '10': 85, '작품': 86, '사랑': 87, '할': 88, '없다': 89, '다시': 90, '하나': 91, '볼': 92, '마지막

## 토치 텍스트의 데이터 로더 만들기

In [15]:
from torchtext.legacy.data import Iterator

In [68]:
# 하이퍼파라미터
BATCH_SIZE = 128
lr = 0.001
EPOCHS = 10

In [69]:
train_loader = Iterator(dataset = train_data, batch_size = batch_size)
test_loader = Iterator(dataset = test_data, batch_size = batch_size)

In [70]:
print('훈련 데이터의 미니 배치 수 : {}'.format(len(train_loader)))
print('테스트 데이터의 미니 배치 수 : {}'.format(len(test_loader)))

훈련 데이터의 미니 배치 수 : 18750
테스트 데이터의 미니 배치 수 : 6250


## RNN 모델 구현

In [71]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchtext import data, datasets
import random

In [72]:
class GRU(nn.Module):
    def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes, dropout_p = 0.5):
        super(GRU, self).__init__()
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim

        self.embed = nn.Embedding(n_vocab, embed_dim)
        self.dropout = nn.Dropout(dropout_p)
        self.gru = nn.GRU(embed_dim, self.hidden_dim,
                          num_layers=self.n_layers,
                          batch_first=True)
        self.out = nn.Linear(self.hidden_dim, n_classes)

    def forward(self, x):
        x = self.embed(x)
        h_0 = self._init_state(batch_size=x.size(0)) # 첫번째 히든 스테이트를 0벡터로 초기화
        x,_ = self.gru(x, h_0) # GRU의 리턴값은 (배치크기, 시퀀스 길이, 은닉 상태의 크기)
        h_t = x[:,-1,:] # (배치 크기, 은닉 상태의 크기)의 텐서로 크기가 변경됨. 즉, 마지막 time-step의 은닉 상태만 가져온다.
        self.dropout(h_t)
        logit = self.out(h_t) # (배치 크기, 은닉 상태의 크기) -> 배치 크기기, 출력층의 크기

        return logit

    def _init_state(self, batch_size=1):
        weight = next(self.parameters()).data
        return weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()

In [105]:
class GRU(nn.Module):
    def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes, dropout_p=0.2):
        super(GRU, self).__init__()
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim

        self.embed = nn.Embedding(n_vocab, embed_dim)
        self.dropout = nn.Dropout(dropout_p)
        self.gru = nn.GRU(embed_dim, self.hidden_dim,
                          num_layers=self.n_layers,
                          batch_first=True)
        self.out = nn.Linear(self.hidden_dim, n_classes)

    def forward(self, x):
        x = self.embed(x)
        h_0 = self._init_state(batch_size=x.size(0)) # 첫번째 히든 스테이트를 0벡터로 초기화
        x, _ = self.gru(x, h_0)  # GRU의 리턴값은 (배치 크기, 시퀀스 길이, 은닉 상태의 크기)
        h_t = x[:,-1,:] # (배치 크기, 은닉 상태의 크기)의 텐서로 크기가 변경됨. 즉, 마지막 time-step의 은닉 상태만 가져온다.
        self.dropout(h_t)
        logit = self.out(h_t)  # (배치 크기, 은닉 상태의 크기) -> (배치 크기, 출력층의 크기)
        return logit

    def _init_state(self, batch_size=1):
        weight = next(self.parameters()).data
        return weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()

In [106]:
vocab_size = len(TEXT.vocab) 
n_classes = 2
print('단어 집합의 크기 : {}'.format(vocab_size))
print('클래스의 개수 : {}'.format(n_classes))

단어 집합의 크기 : 10002
클래스의 개수 : 2


In [168]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GRU(1, 256, vocab_size , 128, n_classes, 0.5).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

RuntimeError: ignored

In [165]:
def train(model, optimizer, train_iter):
    model.train()
    for i, batch in enumerate(train_iter):
        x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        
        #y.data.sub_(1) # 레이블 값을 0과 1로 변환
        optimizer.zero_grad()

        logit = model(x)
        y= torch.tensor(y,dtype = torch.long,device =DEVICE)

        loss = F.cross_entropy(logit,y)
        loss.backward()
        optimizer.step()

In [166]:
def evaluate(model, val_iter):
    """evaluate model"""
    model.eval()
    corrects, total_loss = 0, 0
    for batch in val_iter:
        x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        y= torch.tensor(y,dtype = torch.long,device =DEVICE)
        logit = model(x)
        
        loss = F.cross_entropy(logit, y)
        total_loss += loss.item()
        corrects += (logit.max(1)[1].view(y.size()).data == y.data).sum()
    size = len(val_iter.dataset)
    avg_loss = total_loss / size
    avg_accuracy = 100.0 * corrects / size
    return avg_loss, avg_accuracy

In [ ]:
best_val_loss = None
best_model = []
EPOCHS= 10
for e in range(1, EPOCHS + 1 ):
    train(model, optimizer, train_loader)
    val_loss, val_accuracy = evaluate(model, test_loader)

    print("[Epoch: %d] val loss : %5.2f | val accuracy : %5.2f" % (e, val_loss, val_accuracy))

    # 검증 오차가 가장 적은 최적의 모델을 저장
    if not best_val_loss or val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = model